In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

import utils as utils
from datasets import load_dataset
from typing import Dict

import matplotlib.pyplot as plt

In [ ]:
load_dotenv()
client = OpenAI()

In [ ]:
model_id = 'gpt-3.5-turbo-1106'
dataset_id = 'HuggingFaceH4/no_robots'

In [ ]:
dataset = load_dataset(dataset_id)

In [ ]:
examples = dataset['train_sft']

In [ ]:
def get_num_convo(example: Dict):
    example['num_convo'] = len(example['messages'])
    return example

def get_prompt(
        example: Dict, message_key: str
) -> Dict:
    system_prompt = {
        "role": "system",
        "content": "You are an expert tranlator who traslates given text in English to Devnagri Hindi"
    }

    example_prompts = [
        {
            "role": "user",
            "content": f"Which famous landmarks should I visit in London, beyond the usual ones?"
        },
        {
            "role": "assistant",
            "content": f"लंदन में मुझे कौन से प्रसिद्ध स्थल देखने चाहिए, जो आमतौर पर नहीं होते हैं?"
        },
        {
            "role": "user",
            "content": "Here is an offbeat and lesser-known place in London that locals might recommend: God's Own Junkyard - a neon wonderland filled with vintage and new neon signs. There are many other hidden gems in London, and a quick Google search for ‘offbeat things in London’ will bring up many blogs and resources with more options."
        },
        {
            "role": "assistant",
            "content": " यहां लंदन में एक अनोखा और कम जाना-पहचाना स्थल है जिसकी स्थानीय लोग सिफारिश कर सकते हैं: गॉड्स ओन जंकयार्ड - एक नियॉन वंडरलैंड जो पुराने और नए नियॉन साइन्स से भरा हुआ है। लंदन में कई अन्य छुपे हुए रत्न हैं, और ‘लंदन में अनोखी चीजें’ के लिए गूगल सर्च करने पर आपको कई ब्लॉग्स और संसाधन मिल जाएंगे जिनमें और भी विकल्प होंगे।"
        }
    ]

    prompts_to_insert = [
        system_prompt
    ] + example_prompts

    return prompts_to_insert

    # for m in example[message_key]:
    #     prompts_to_insert += [{
    #         "role": "user",
    #         "content": m['content']
    #     }]

    #     yield prompts_to_insert
    #     _ = prompts_to_insert.pop()

In [ ]:
t = examples.map(
    lambda x: utils.get_stringify_conversation(x, message_key='messages'),
    num_proc=20
).map(
    get_num_convo,
    num_proc=20
)

In [ ]:
m = t.filter(lambda x: x['num_convo'] > 5)

In [ ]:
init_prompt = get_prompt(m[0], 'messages')

In [ ]:
a = {
    'role': 'user',
    'content': m[0]['messages'][3]['content']
}

In [ ]:
a

In [ ]:
o = client.chat.completions.create(
    model=model_id,
    messages=init_prompt+[a]
)

In [ ]:
o.choices[0].message.content

In [ ]:
init_prompt

In [ ]:
f = get_prompt(m[0], 'messages')

In [ ]:
from tqdm import tqdm

In [ ]:
ans = []
for prompt in tqdm(f, total=len(m[0]['messages'])):
    completion = client.chat.completions.create(
        model=model_id,
        messages=prompt
    )

    ans.append(completion)

In [ ]:
prompt

In [ ]:
m[0]

In [ ]:
for a in ans:
    print(a.usage.total_tokens, a.choices[0].message.content)

In [ ]:
ans[0]

In [ ]:
for e in messages:
    print(e['content'])

In [ ]:
completion = client.chat.completions.create(
    model=model_id,
    messages=messages
)

In [ ]:
print(completion.choices)

In [ ]:
print(completion.usage)

In [ ]:
print(completion.choices[0].message.content)